In [ ]:
pip install neo4j

In [ ]:
pip install pandas

In [34]:
import pandas as pd
import os

In [35]:
from neo4j import GraphDatabase

# Replace these with your actual AuraDB connection details
uri = "neo4j+s://b480f347.databases.neo4j.io"
username = "neo4j"
password = "8erltLOKbpg68_LbDjbZtsHQC-nAKV-advvPvHLdpII"

driver = GraphDatabase.driver(uri, auth=(username,password))


In [ ]:
from google.colab import drive  # Mounting drive to Google Colab. Ignore if not using Colab
drive.mount('/content/drive')

In [36]:
def create_nodes(tx, data, label, identifier_key):
    for index, row in data.iterrows():
        node_data = dict(row)
        tx.run(
            f"CREATE (node:{label} {{{identifier_key}: $value}})",
            value=node_data[identifier_key]
        )

def create_nodes_packages(tx, data, label, identifier_key):
    for index, row in data.iterrows():
        node_data = dict(row)
        # Create the node with properties from the CSV columns
        query = (
            f"CREATE (node:{label} {{"
            f"{identifier_key}: $value, "
            f"SPDXID: $SPDXID, "
            f"PackageName: $PackageName, "
            f"PackageDownloadLocation: $PackageDownloadLocation, "
            f"FilesAnalyzed: $FilesAnalyzed, "
            f"PackageLicenseConcluded: $PackageLicenseConcluded, "
            f"PackageLicenseDeclared: $PackageLicenseDeclared, "
            f"PackageCopyrightText: $PackageCopyrightText, "
            f"PackageChecksum: $PackageChecksum"
            # Add more properties as needed, and remove the trailing comma
            f"}})"
        )
        tx.run(query, {
            'value': node_data[identifier_key],
            'SPDXID': node_data['SPDXID'],
            'PackageName': node_data['PackageName'],
            'PackageDownloadLocation': node_data['PackageDownloadLocation'],
            'FilesAnalyzed': node_data['FilesAnalyzed'],
            'PackageLicenseConcluded': node_data['PackageLicenseConcluded'],
            'PackageLicenseDeclared': node_data['PackageLicenseDeclared'],
            'PackageCopyrightText': node_data['PackageCopyrightText'],
            'PackageChecksum': node_data['PackageChecksum']
            # Add more property values as needed
        })

def create_nodes_spdx(tx, data, label, identifier_key):
    for index, row in data.iterrows():
        node_data = dict(row)
        # Create the node with properties from the CSV columns
        query = (
            f"CREATE (node:{label} {{"
            f"{identifier_key}: $value, "
            f"SPDXID: $SPDXID, "
            f"FileName: $FileName, "
            f"FileChecksum: $FileChecksum, "
            f"LicenseConcluded: $LicenseConcluded, "
            f"LicenseInfoInFile: $LicenseInfoInFile, "
            f"FileCopyrightText: $FileCopyrightText"
            # Add more properties as needed, and remove the trailing comma
            f"}})"
        )
        tx.run(query, {
            'value': node_data[identifier_key],
            'SPDXID': node_data['SPDXID'],
            'FileName': node_data['FileName'],
            'FileChecksum': node_data['FileChecksum'],
            'LicenseConcluded': node_data['LicenseConcluded'],
            'LicenseInfoInFile': node_data['LicenseInfoInFile'],
            'FileCopyrightText': node_data['FileCopyrightText']
            # Add more property values as needed
        })

In [42]:
# Specify the root directory containing subfolders
root_directory = '/content/drive/MyDrive/Cloud Computing EC528/test'

# Iterate through subfolders
for folder_name in os.listdir(root_directory):
    folder_path = os.path.join(root_directory, folder_name)

    if os.path.isdir(folder_path):
        print(f"Processing folder: {folder_name}")

        driver = GraphDatabase.driver(uri, auth=(username,password))

        # Load the CSV files for this folder
        packages_data = pd.read_csv(os.path.join(folder_path, f'{folder_name}_packages_output.csv'))
        spdx_data = pd.read_csv(os.path.join(folder_path, f'{folder_name}_spdx_output.csv'))
        relationships_data = pd.read_csv(os.path.join(folder_path, f'{folder_name}_relationships_output.csv'))

        """
        # Create a node for the folder and connect all nodes to it
        with driver.session() as session:
            print(f"Creating folder node for: {folder_name}")
            session.run("CREATE (folder:Folder {name: $folder_name})", folder_name=folder_name)
        """
        """
        with driver.session() as session:
            # Connect all nodes in this subfolder to the folder
            for label, identifier_key in [('SPDXNode', 'SPDXID'), ('PackageNode', 'SPDXID')]:
                print(f"Connecting {label} nodes to folder node for: {folder_name}")
                session.run(
                    f"MATCH (node:{label}) WHERE NOT node:Folder "
                    "WITH node "
                    "MATCH (folder:Folder {name: $folder_name}) "
                    "CREATE (node)-[:BELONGS_TO]->(folder)",
                    label=label,
                    folder_name=folder_name
                )
        """

        with driver.session() as session:
            # Create nodes and relationships for this subfolder
            print(f"Creating nodes and relationships for: {folder_name}")
            session.write_transaction(create_nodes, spdx_data, 'SPDXNode', 'SPDXID')
            session.write_transaction(create_nodes, packages_data, 'PackageNode', 'SPDXID')

            for index, row in relationships_data.iterrows():
                relationship_data = dict(row)
                session.run(
                    "MATCH (source {SPDXID: $source}), (target {SPDXID: $target}) "
                    "CREATE (source)-[:RELATIONSHIP_TYPE {type: $types}]->(target)",
                    source=relationship_data['RelationshipType'],
                    target=relationship_data['Target'],
                    types=relationship_data['Source']
                )

        # Create a node for the folder and connect all nodes to it
        with driver.session() as session:
            print(f"Creating folder node for: {folder_name}")
            session.run("CREATE (folder:Folder {name: $folder_name})", folder_name=folder_name)

        with driver.session() as session:
            # Connect all nodes in this subfolder to the folder
            for label, identifier_key in [('SPDXNode', 'SPDXID'), ('PackageNode', 'SPDXID')]:
                print(f"Connecting {label} nodes to folder node for: {folder_name}")
                session.run(
                    f"MATCH (node:{label}) WHERE NOT node:Folder "
                    "WITH node "
                    "MATCH (folder:Folder {name: $folder_name}) "
                    "CREATE (node)-[:BELONGS_TO]->(folder)",
                    label=label,
                    folder_name=folder_name
                )




Processing folder: v1.18.20
Creating nodes and relationships for: v1.18.20


<ipython-input-42-10728350d05d>:42: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes, spdx_data, 'SPDXNode', 'SPDXID')
<ipython-input-42-10728350d05d>:43: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes, packages_data, 'PackageNode', 'SPDXID')


Creating folder node for: v1.18.20
Connecting SPDXNode nodes to folder node for: v1.18.20
Connecting PackageNode nodes to folder node for: v1.18.20
Processing folder: v1.19.13
Creating nodes and relationships for: v1.19.13


<ipython-input-42-10728350d05d>:42: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes, spdx_data, 'SPDXNode', 'SPDXID')
<ipython-input-42-10728350d05d>:43: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes, packages_data, 'PackageNode', 'SPDXID')


Creating folder node for: v1.19.13
Connecting SPDXNode nodes to folder node for: v1.19.13
Connecting PackageNode nodes to folder node for: v1.19.13
Processing folder: v1.19.12
Creating nodes and relationships for: v1.19.12


<ipython-input-42-10728350d05d>:42: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes, spdx_data, 'SPDXNode', 'SPDXID')
<ipython-input-42-10728350d05d>:43: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes, packages_data, 'PackageNode', 'SPDXID')


Creating folder node for: v1.19.12
Connecting SPDXNode nodes to folder node for: v1.19.12
Connecting PackageNode nodes to folder node for: v1.19.12
